# Function calling

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field

### Chat

Previously used Chat LLM

```python
chat_llm = ChatOpenAI(
    base_url="http://localhost:9999/v1",
    api_key="dev",
    model="mistral",
    temperature=0,
    max_tokens=128,
)
```

### Function Calling

In [ ]:
class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class SendEmail(BaseModel):
    """Send an email to receiver."""

    receiver: str = Field(..., description="Email receiver")

In [ ]:
function_calling_llm = ChatOpenAI(
    base_url="http://localhost:9999/v1",
    api_key="dev",
    model="mistral-function-calling",
    temperature=0,
    max_tokens=128,
)
tools = [Multiply, SendEmail]
function_calling_llm = function_calling_llm.bind_tools(tools)
function_calling_chain = function_calling_llm | JsonOutputToolsParser()

In [ ]:
result = function_calling_chain.invoke("What is 3 * 12?")
result

In [ ]:
mul = Multiply(**result[0]['args']['input'])
mul.a * mul.b

In [ ]:
result = function_calling_chain.invoke("Send an email to Adam Księżyk")
result

In [ ]:
email = SendEmail(**result[0]['args'])
print(f"Email sent to {email.receiver!r}")

### Try to confuse it

In [ ]:
result = function_calling_chain.invoke("""Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.
Esse cillum dolore eu fugiat nulla pariatur. Send an email to Adam Księżyk. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit
anim id est laborum.""")
result